In [1]:
%matplotlib inline


# NEON Data

This example shows how to download data from
NEON and ARM 2m surface meteorology stations
on the North Slope and plot them


In [ ]:
%pip install act-atmos

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr

import act

# Set up plot space ahead of time
fig, ax = plt.subplots(3, figsize=(10, 7))

# Plotting up high-temporal resolution 2D data can be very slow at times.
# In order to increase the speed, the data can be resampled to a courser
# resolution prior to plotting.  Using Xarray's resample and selecting
# the nearest neighbor will greatly increase the speed.
from netCDF4 import MFDataset

obj = act.io.armfiles.read_netcdf('maomwrlosM1.b1.20151117.000031.cdf')

obj = obj.resample(time='5min').nearest()
#print(obj.variables)

#print(obj)

# These data can be plotted up using the existing xarray functionality
# which is quick and easy
obj['sky31'].plot(x='time', ax=ax[0])

# or using ACT
display = act.plotting.TimeSeriesDisplay(obj)
display.assign_to_figure_axis(fig, ax[1])
display.plot('sky31')

# When using ACT, the axis object can also be manipulated using normal
# matplotlib calls for more personalized customizations
display = act.plotting.TimeSeriesDisplay(obj)
display.assign_to_figure_axis(fig, ax[2])
display.plot('sky31')
display.axes[-1].set_ylim([0, 1500])

plt.show()

In [ ]:
# plot_temperature.py
# Example code to plot ARM NetCDF data
# First import the NetCDF4 package
from netCDF4 import Dataset
from netCDF4 import MFDataset

# Next import plotting functions
from pylab import *
# Load the NetCDF file into a data object
#data = MFDataset('/home/maou/Área de Trabalho/Victor/Estágio/Dados de Satelite/*.09.2015 Antartica.nc', mode='r') **#Aqui eu gostaria de colocar outros 29 dados e não sei como.**

#nc_file = Dataset('maomwrlosM1.b1.20151117.000031.cdf','r',format='NETCDF4_CLASSIC')
nc_file = MFDataset('L:/GitHub/CAP-Dissertation-Complementary-studies/Notebooks/data/maomwrlosM1.b1.*.cdf','r')
#print(nc_file.variables)
# Load the time and temperature as data objects

temp_obj = nc_file.variables['sky31']
time_obj = nc_file.variables['time']
# Load the time and temperature data into the workspace

temperature = temp_obj[:]
time = time_obj[:]
# Convert the time from seconds since midnight GMT to hours since midnight GMT
time_hour = time/3600
# Now plot the data
# Setup the plot

display = plot(figsize=(15, 10))

# Plot temperature data in top plot


plot(time_hour, temperature)
# Limit the axes ranges
xlim(0,24)
# You can also force the ticks to be at 6-hour intervals (for example)
xticks([0,6,12,18,24])
# Add axes labels and a title
xlabel("Hours Since Midnight")
ylabel("Temperature (Degrees C)")
title("Temperature at Barrow: February 10, 2013")
# Output as a PNG format file
savefig('testplot.png', format='png')
# When plotting from within a script you have to instruct the script to display the output
show()

print(nc_file.variables)

In [ ]:
import act
import matplotlib.pyplot as plt

# Place your username and token here
username = 'LourencoCavalcante' 
token = '311d84bc76093a97' 

# Download AERIoe data and ceilometer data
#act.discovery.download_data('LourencoCavalcante' , '311d84bc76093a97', 'sgpaerioe1turnC1.c1', '2022-02-11', '2022-02-11') 
#act.discovery.download_data('LourencoCavalcante' , '311d84bc76093a97', 'sgpceilC1.b1', '2022-02-11', '2022-02-11') 

# Read in AERIoe and ceilometer data
aerioe_ds = act.io.armfiles.read_netcdf('sgpaerioe1turnC1.c1/sgpaerioe1turnC1.c1.20220211.000354.nc')
ceil_ds = act.io.armfiles.read_netcdf('sgpceilC1.b1/sgpceilC1.b1.20220211.000005.nc')

# There isn't information content from the AERI above 3 km
# Remove data with a height above 3 km
aerioe_ds = aerioe_ds.sel(height=aerioe_ds.coords['height'] <= 3)

# Convert Ceilometer cloud base height to km
ceil_ds['first_cbh'] = ceil_ds['first_cbh']/1000

# Remove first_cbh if it is higher than 3 km
ceil_ds['first_cbh'] = ceil_ds['first_cbh'][~(ceil_ds['first_cbh'] > 3)]

# Create a TimeSeriesDisplay object
display = act.plotting.TimeSeriesDisplay(
   {'AERIoe': aerioe_ds, 'Ceilometer': ceil_ds},
   subplot_shape=(2,), figsize=(15,10)
)

# Plot data
display.plot('first_cbh', dsname='Ceilometer', marker='+', color='black', markeredgewidth=3,
   linewidth=0, subplot_index=(0,), label='cbh')
display.plot('temperature', dsname='AERIoe', cmap='viridis', set_shading='nearest',
   add_nan=True, subplot_index=(0,))

display.plot('first_cbh', dsname='Ceilometer', marker='+', color='black', markeredgewidth=3,
   linewidth=0, subplot_index=(1,), label='cbh')
display.plot('waterVapor', dsname='AERIoe', cmap='act_HomeyerRainbow', set_shading='nearest',
   add_nan=True, subplot_index=(1,))

plt.savefig('sgpaerioe1turnC1.c1.20220211.png')